In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data', header=None) 
X = df.iloc[:, 2:].values
y = df.iloc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe_svc = make_pipeline(StandardScaler(), SVC(random_state=42))
param_range = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = [{'svc__C': param_range, 'svc__kernel': ['linear']}, {'svc__C': param_range, 'svc__gamma': param_range, 'svc__kernel': ['rbf']}]
# scoring - метрика, на основе которой будет оцениваться качество модели, cv - блоки при перекрёстной проверке, refit - модель переобучится на всём наборе данных после подбора гиперпараметров
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=-1, refit=True)

gs = gs.fit(X_train, y_train)
print(gs.best_params_)
print(gs.best_score_)

{'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
0.9757004830917875


In [10]:
clf = gs.best_estimator_

In [11]:
import pickle
import os


# создаём каталог(для приложения) и подкаталог(для сериализованных объектов)
dest = os.path.join('cancel_classifier', 'model.pkl')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pcl'), 'wb'), protocol=4) # консервируем clf в указанный путь с наилучшим видом протокола. Если не работает, то попробовать протокол поменьше.